This notebook runs the code in bagging_models.py for uncertainty quantification.

In [1]:
'''First, we need to define the path of where to get the dataset, and define other parameters that we will need'''
import sys
sys.path.append('/Users/jakehirst/Desktop/sfx/sfx_ML_code/sfx_ML/New_Models')

from Bagging_models import *
from ReCalibration import *
from Backward_feature_selection import *
import ast

model_types = ['ANN', 'RF', 'GPR','ridge']
# model_types = ['ridge']
all_labels = ['height', 'phi', 'theta', 
                            'impact site x', 'impact site y', 'impact site z', 
                            'impact site r', 'impact site phi', 'impact site theta']

all_labels = ['height', 
                            'impact site x', 'impact site y', 'impact site z', 
                            'impact site r', 'impact site phi', 'impact site theta']

num_models_list = [20]
labels_to_predict = ['impact site x', 'impact site y', 'height']
labels_to_predict = ['height']

with_or_without_transformations = 'with'
with_or_without_transformations = 'without'

Paper2_path = f'/Volumes/Jake_ssd/Paper 2/{with_or_without_transformations}_transformations'
Paper2_path = f'/Volumes/Jake_ssd/with_phi_theta'
if(not os.path.exists(Paper2_path)): os.makedirs(Paper2_path)
model_folder = Paper2_path + f'/UQ_bagging_models_{with_or_without_transformations}_transformations'
data_folder = Paper2_path + '/5fold_datasets'
results_folder = Paper2_path + '/Compare_Code_5_fold_ensemble_results'
# hyperparam_folder = Paper2_path + f'/bayesian_optimization_{with_or_without_transformations}_transformations'
hyperparam_folder = f'/Volumes/Jake_ssd/Paper 2/{with_or_without_transformations}_transformations' + f'/bayesian_optimization_{with_or_without_transformations}_transformations'


image_folder = '/Users/jakehirst/Desktop/sfx/sfx_ML_data/images_sfx/new_dataset/Visible_cracks'

if(with_or_without_transformations == 'with'):
    full_dataset_pathname = "/Volumes/Jake_ssd/Paper 1/Paper_1_results_WITH_feature_engineering/dataset/feature_transformations_2023-11-16/height/HEIGHTALL_TRANSFORMED_FEATURES.csv"
    backward_feat_selection_results_folder = '/Volumes/Jake_ssd/Paper 1/Paper_1_results_WITH_feature_engineering/results'
else:
    # full_dataset_pathname = "/Volumes/Jake_ssd/Paper 1/Paper_1_results_no_feature_engineering/dataset/New_Crack_Len_FULL_OG_dataframe_2023_11_16.csv"
    full_dataset_pathname = "/Volumes/Jake_ssd/Paper 2/New_Crack_Len_FULL_OG_dataframe_2024_02_22.csv"
    backward_feat_selection_results_folder = Paper2_path + '/Paper_2_results_WITHOUT_feature_engineering/results' 
    df = pd.read_csv(full_dataset_pathname, index_col=0)
    all_features = df.columns
    all_features = all_features.drop(all_labels)
    all_features = str(all_features.drop('timestep_init').to_list())

    print(all_features)
 


'''Only have to uncomment this if the 5 fold datasets have not been made or need to be remade'''
# make_5_fold_datasets(data_folder, full_dataset_pathname, image_folder)


print('ALL_TRANSFORMED_FEATURES' in full_dataset_pathname)


forestci is an optional dependency. To install latest forestci compatabilty with scikit-learn>=0.24, run pip install git+git://github.com/scikit-learn-contrib/forest-confidence-interval.git
XGBoost is an optional dependency. If you want to use XGBoost models, please manually install xgboost package with pip install xgboost. If have error with finding libxgboost.dylib library, dobrew install libomp. If do not have brew on your system, first do ruby -e "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/master/install)" from the Terminal
scikit-lego is an optional dependency, enabling use of the LowessRegression model. If you want to use this model, do "pip install scikit-lego"
['init z', 'init y', 'init x', 'max_prop_speed', 'avg_prop_speed', 'dist btw frts', 'crack len', 'linearity', 'max thickness', 'mean thickness', 'median_thickness', 'var_thickness', 'std_thickness', 'thickness_at_init', 'max_kink', 'abs_val_mean_kink', 'mean_kink', 'sum_kink', 'abs_val_sum_kink', 'med

In [2]:
'''get the appropriate features that each model will use based on backward feature elimination'''
all_features_to_keep = {}

min_features = 1 #minimum number of features you want to select from BFS (backward feature selection)
max_features = 25 #maximum number of features you want to select from BFS
for label in labels_to_predict:
    all_features_to_keep[label] = {}
    for model_type in model_types:
        
        if('ALL_TRANSFORMED_FEATURES' in full_dataset_pathname):
            print('true')
        #TODO use code below if using feature selection
            best_features = get_best_features(backward_feat_selection_results_folder, label, model_type, min_features, max_features)
            all_features_to_keep[label][model_type] = best_features
        
        else:
            print('using just the basic features')
            #TODO use code below if NOT using feature selection
            all_features_to_keep[label][model_type] = all_features

print(all_features_to_keep)
        

using just the basic features
using just the basic features
using just the basic features
using just the basic features
{'height': {'ANN': "['init z', 'init y', 'init x', 'max_prop_speed', 'avg_prop_speed', 'dist btw frts', 'crack len', 'linearity', 'max thickness', 'mean thickness', 'median_thickness', 'var_thickness', 'std_thickness', 'thickness_at_init', 'max_kink', 'abs_val_mean_kink', 'mean_kink', 'sum_kink', 'abs_val_sum_kink', 'median_kink', 'std_kink', 'var_kink', 'avg_ori', 'angle_btw', 'phi', 'theta']", 'RF': "['init z', 'init y', 'init x', 'max_prop_speed', 'avg_prop_speed', 'dist btw frts', 'crack len', 'linearity', 'max thickness', 'mean thickness', 'median_thickness', 'var_thickness', 'std_thickness', 'thickness_at_init', 'max_kink', 'abs_val_mean_kink', 'mean_kink', 'sum_kink', 'abs_val_sum_kink', 'median_kink', 'std_kink', 'var_kink', 'avg_ori', 'angle_btw', 'phi', 'theta']", 'GPR': "['init z', 'init y', 'init x', 'max_prop_speed', 'avg_prop_speed', 'dist btw frts', 'cr

In [3]:
'''Now we will make all of the bagging models'''
for fold_no in range(1,6):
    for model_type in model_types:
        for label_to_predict in labels_to_predict:
            for num_models in num_models_list:
                
                print(f'\n$$$$$$$$$$$$$$$$$$$$$$$$$$$$$\ Predicting {label_to_predict} using {model_type} $$$$$$$$$$$$$$$$$$$$$$$$$$$$$\n')
                
                all_labels = ['height', 'phi', 'theta', 
                            'impact site x', 'impact site y', 'impact site z', 
                            'impact site r', 'impact site phi', 'impact site theta']
                
                all_labels = ['height',
                            'impact site x', 'impact site y', 'impact site z', 
                            'impact site r', 'impact site phi', 'impact site theta']

                print(f'{data_folder}/{label_to_predict}/fold{fold_no}/train_features.csv')
                training_features = pd.read_csv(f'{data_folder}/{label_to_predict}/fold{fold_no}/train_features.csv').reset_index(drop=True)
                training_labels = pd.read_csv(f'{data_folder}/{label_to_predict}/fold{fold_no}/train_labels.csv').reset_index(drop=True)

                model_saving_folder = f'{model_folder}/{label_to_predict}/{model_type}/{num_models}_models/fold_{fold_no}'
                if(not os.path.exists(model_saving_folder)):
                    os.makedirs(model_saving_folder)
                    
                results_saving_folder = f'{results_folder}/{label_to_predict}/{model_type}/{num_models}_models/fold_{fold_no}'
                if(not os.path.exists(results_saving_folder)):
                    os.makedirs(results_saving_folder)
                # make_dirs(model_saving_folder)
                # make_dirs(results_saving_folder)

                '''TODO gotta find out what features to use for each label before testing on new dataset'''
                features_to_keep = ast.literal_eval(all_features_to_keep[label_to_predict][model_type])
                print(features_to_keep)
                make_linear_regression_models_for_ensemble(training_features, training_labels, model_saving_folder, label_to_predict, num_models, features_to_keep, hyperparam_folder, model_type=model_type)




$$$$$$$$$$$$$$$$$$$$$$$$$$$$$\ Predicting height using ANN $$$$$$$$$$$$$$$$$$$$$$$$$$$$$

/Volumes/Jake_ssd/with_phi_theta/5fold_datasets/height/fold1/train_features.csv
['init z', 'init y', 'init x', 'max_prop_speed', 'avg_prop_speed', 'dist btw frts', 'crack len', 'linearity', 'max thickness', 'mean thickness', 'median_thickness', 'var_thickness', 'std_thickness', 'thickness_at_init', 'max_kink', 'abs_val_mean_kink', 'mean_kink', 'sum_kink', 'abs_val_sum_kink', 'median_kink', 'std_kink', 'var_kink', 'avg_ori', 'angle_btw', 'phi', 'theta']
working on model 0
patience reached
End train R2 score = 0.3472033100059535 validation R2 score = 0.09053422734504701
working on model 1
patience reached
End train R2 score = 0.33891903714210225 validation R2 score = 0.34025627566900807
working on model 2
patience reached
End train R2 score = 0.3159167932850806 validation R2 score = 0.12432256152780852
working on model 3
patience reached
End train R2 score = 0.40751584440932065 validation R2 score 

In [5]:
'''Now we will evaluate the performance of the bagging models'''


for model_type in model_types:
    print(f'MODEL TYPE = {model_type}')
    for label_to_predict in labels_to_predict:
        print(f'LABEL = {label_to_predict}')
        for num_models in num_models_list:
            performance_data = []
            for fold_no in range(1,6):
                print(f'fold {fold_no}')

                #defining folders to get the models and to store the results
                model_saving_folder = f'{model_folder}/{label_to_predict}/{model_type}/{num_models}_models/fold_{fold_no}'
                results_saving_folder = f'{results_folder}/{label_to_predict}/{model_type}/{num_models}_models/fold_{fold_no}'
                
                #defining folders where the datasets are coming from (5-fold cv)
                test_features_path = Paper2_path + f'/5fold_datasets/{label_to_predict}/fold{fold_no}/test_features.csv'
                test_labels_path = Paper2_path + f'/5fold_datasets/{label_to_predict}/fold{fold_no}/test_labels.csv'
                train_features_path = Paper2_path + f'/5fold_datasets/{label_to_predict}/fold{fold_no}/train_features.csv'
                train_labels_path = Paper2_path + f'/5fold_datasets/{label_to_predict}/fold{fold_no}/train_labels.csv'

                #defining the features that each model used (since they vary with each model)
                features_to_keep = ast.literal_eval(all_features_to_keep[label_to_predict][model_type])
                
                #predicting the test and train sets with the bagging models
                test_r2, test_ensemble_predictions, test_ensemble_uncertanties, test_labels = Get_predictions_and_uncertainty_with_bagging(test_features_path, test_labels_path, model_saving_folder, results_saving_folder, features_to_keep, label_to_predict, model_type)
                train_r2, train_ensemble_predictions, train_ensemble_uncertanties, train_labels = Get_predictions_and_uncertainty_with_bagging(train_features_path, train_labels_path, model_saving_folder, results_saving_folder, features_to_keep, label_to_predict, model_type)

                #defining the residual errors of the predictions
                train_labels_arr = train_labels
                train_predictions_arr = np.array(train_ensemble_predictions)
                test_labels_arr = test_labels
                test_predictions_arr = np.array(test_ensemble_predictions)
                train_residuals = pd.Series(np.abs(train_labels_arr - train_predictions_arr))
                test_residuals = pd.Series(np.abs(test_labels_arr - test_predictions_arr))

                a = 0
                b = 0
                '''getting calibration factors *** linear'''
                # cf = CorrectionFactors(train_residuals, pd.Series(train_ensemble_uncertanties))
                # a, b = cf.nll()
                # print(f'a = {a} b = {b}')
                # calibrated_train_uncertainties = pd.Series(a * np.array(train_ensemble_uncertanties) + b, name='train_model_errors')
                # calibrated_test_uncertainties = pd.Series(a * np.array(test_ensemble_uncertanties) + b, name='test_model_errors')
                '''getting calibration factors *** Nonlinear'''
                # a, b = get_calibration_factors(train_residuals, train_ensemble_uncertanties)
                # print(f'a = {a} b = {b}')
                # calibrated_train_uncertainties = pd.Series(a * (train_ensemble_uncertanties**((b/2) + 1)), name='train_model_errors')
                # calibrated_test_uncertainties = pd.Series(a * (test_ensemble_uncertanties**((b/2) + 1)), name='test_model_errors')


                '''
                Calculating and plotting performance metrics as outlined in section 2.3 of Tran et al. (https://dx.doi.org/10.1088/2632-2153/ab7e1a)
                
                Models should be compared in terms of 
                1st - accuracy (R^2) 
                2nd - calibration (miscalibration area)
                3rd - sharpness
                4th - dispersion
                '''
                #CALIBRATION plots and miscalibration area
                #This tells us how 'honest' our uncertainty values are. A perfect calibration plot would mean for a given confidence interval in our prediction
                #(say 90%), we can expect with 90% certainty that the true value falls within that confidence interval.
                miscalibration_area, calibration_error = make_calibration_plots(model_type, test_predictions_arr, test_labels_arr, test_ensemble_uncertanties, results_saving_folder)
                
                #SHARPNESS plots and value
                #Models can be calibrated, but all have very dull uncertainty values (they all have large uncertainties). To ensure UQ is meaningful, models
                #should a be sharp (i.e. uncertainties should be as small as possible.)
                #Sharpness is essentially calculated as the average of predicted standard deviations. #COMMENT Low sharpness values are better.
                stdevs = np.array(test_ensemble_uncertanties)/2 #right now, i multiply the stds by 2 to make it look better in parity plots... but this needs the raw std.
                sharpness, dispersion = plot_sharpness_curve(stdevs, results_saving_folder)
                #DISPERSION value
                #Models can be calibrated and sharp, but even so, if they are all similar uncertainties, then this does not tell us much. To ensure more 
                #meaningful UQ, having a large dispersion of uncertainties is valuable. 
                #Dispersion is calculated using equation 4 of the paper, which is called the coefficient of variation (Cv). #COMMENT High dispersion (Cv) values are better.
                
                
                
                
                
                # blank_model_for_plot = SklearnModel('RandomForestRegressor')
                # mastml_RVE = Error()

                # mastml_RVE.plot_real_vs_predicted_error_uncal_cal_overlay(savepath=results_saving_folder, 
                #                                                         model=blank_model_for_plot, 
                #                                                         data_type='train', 
                #                                                         model_errors=pd.Series(train_ensemble_uncertanties) ,
                #                                                         model_errors_cal= calibrated_train_uncertainties,
                #                                                         residuals= train_residuals, 
                #                                                         dataset_stdev=np.std(train_labels.to_numpy()), 
                #                                                         show_figure=False,
                #                                                         well_sampled_number=0.025)
                
                
                # mastml_RVE.plot_real_vs_predicted_error_uncal_cal_overlay(savepath=results_saving_folder, 
                #                                                         model=blank_model_for_plot, 
                #                                                         data_type='test', 
                #                                                         model_errors=pd.Series(test_ensemble_uncertanties) ,
                #                                                         model_errors_cal= calibrated_test_uncertainties,
                #                                                         residuals= test_residuals, 
                #                                                         dataset_stdev=np.std(train_labels.to_numpy()), 
                #                                                         show_figure=False,
                #                                                         well_sampled_number=0.025)
                
                '''using library from Palmer et al. to make an rve plot'''
                # train_intercept, train_slope, CAL_train_intercept, CAL_train_slope, train_intercept, test_slope, CAL_test_intercept, CAL_test_slope = make_RVE_plots(label_to_predict, model_type, test_ensemble_predictions, test_ensemble_uncertanties, test_labels, train_ensemble_predictions, train_ensemble_uncertanties, train_labels, results_saving_folder, num_bins=15)
                
                '''collecting the performance data from this model'''
                # performance_data.append([15, fold_no, train_r2, test_r2, a, b, train_intercept, train_slope, CAL_train_intercept, CAL_train_slope, train_intercept, test_slope, CAL_test_intercept, CAL_test_slope, miscalibration_area, calibration_error])
                performance_data.append([fold_no, train_r2, test_r2, miscalibration_area, calibration_error, sharpness, dispersion])

            # columns = ['num bins', 'fold_no', 'train R2', 'test R2',  'a', 'b', 'train_intercept', 'train_slope', 'CAL_train_intercept', 'CAL_train_slope', 'train_intercept', 'test_slope', 'CAL_test_intercept', 'CAL_test_slope', 'miscal_area', 'cal_error']
            columns = ['fold_no', 'train R2', 'test R2', 'miscal_area', 'cal_error', 'sharpness', 'dispersion']
            df = pd.DataFrame(columns=columns)
            for row in performance_data:
                df.loc[len(df)] = row
            average_row = df.mean()
            df = df.append(average_row, ignore_index=True)
                
            results_saving_folder = f'{results_folder}/{label_to_predict}/{model_type}/{num_models}_models'
            df.to_csv(results_saving_folder + f'/{label_to_predict}_{model_type}_{num_models}results.csv', index=False)

MODEL TYPE = ANN
LABEL = height
fold 1
Calibration error = 3.87
fold 2
Calibration error = 2.89
fold 3
Calibration error = 1.65
fold 4
Calibration error = 3.83
fold 5
Calibration error = 3.47
MODEL TYPE = RF
LABEL = height
fold 1
Calibration error = 4.92
fold 2
Calibration error = 5.04
fold 3
Calibration error = 4.42
fold 4
Calibration error = 5.12
fold 5
Calibration error = 5.32
MODEL TYPE = GPR
LABEL = height
fold 1
Calibration error = 9.83
fold 2
Calibration error = 4.82
fold 3
Calibration error = 5.49
fold 4
Calibration error = 6.87
fold 5
Calibration error = 6.84
MODEL TYPE = ridge
LABEL = height
fold 1
Calibration error = 5.66
fold 2
Calibration error = 4.20
fold 3
Calibration error = 2.65
fold 4
Calibration error = 5.70
fold 5
Calibration error = 5.36
